# Reflection Pattern

The first pattern we are going to implement is the **reflection pattern**. 

---

<img src="../img/reflection_pattern.png" alt="Alt text" width="600"/>

---

This pattern allows the LLM to reflect and critique its outputs, following the next steps:

1. The LLM **generates** a candidate output. If you look at the diagram above, it happens inside the **"Generate"** box.
2. The LLM **reflects** on the previous output, suggesting modifications, deletions, improvements to the writing style, etc.
3. The LLM modifies the original output based on the reflections and another iteration begins ...

**Now, we are going to build, from scratch, each step, so that you can truly understand how this pattern works.**

## Generation Step

The first thing we need to consider is:

> What do we want to generate? A poem? An essay? Python code?

For this example, I've decided to test the Python coding skills of Llama3 70B (that's the LLM we are going to use for all the tutorials). In particular, we are going to ask our LLM to code a famous sorting algorithm: **Merge Sort**. 

---

<img src="../img/mergesort.png" alt="Alt text" width="500"/>

### Groq Client and relevant imports

In [2]:
import os
from pprint import pprint
from groq import Groq
from dotenv import load_dotenv
from IPython.display import display_markdown

# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

client = Groq()

We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python 
programmer eager to receive feedback / critique by the user.

In [4]:
generation_chat_history = [
    {
        "role": "system",
        "content": "You are a Python programmer tasked with generating high quality Python code."
        "Your task is to Generate the best content possible for the user's request. If the user provides critique," 
        "respond with a revised version of your previous attempt."
    }
]

Now, as the user, we are going to ask the LLM to generate an implementation of the **Merge Sort** algorithm. Just add a new message with the **user** role to the chat history.

In [5]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of the Merge Sort algorithm"
    }
)

Let's generate the first version of the essay.

In [6]:
mergesort_code = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

generation_chat_history.append(
    {
        "role": "assistant",
        "content": mergesort_code
    }
)

In [7]:
display_markdown(mergesort_code, raw=True)

Here's a Python implementation of the Merge Sort algorithm:

```Python
def merge_sort(arr):
    # If the input array contains fewer than two elements,
    # then return it as it's already sorted
    if len(arr) < 2:
        return arr

    # Find the middle point and devide it into two halves
    middle = len(arr) // 2
    left_half = arr[:middle]
    right_half = arr[middle:]

    # Recursively call merge_sort on the two halves
    left_half = merge_sort(left_half)
    right_half = merge_sort(right_half)

    # Merge the two sorted halves into a single sorted array
    return merge(left_half, right_half)


def merge(left, right):
    merged = []
    left_index = 0
    right_index = 0

    # Merge smaller elements first
    while left_index < len(left) and right_index < len(right):
        if left[left_index] < right[right_index]:
            merged.append(left[left_index])
            left_index += 1
        else:
            merged.append(right[right_index])
            right_index += 1

    # If there are remaining elements in either half, append them to the merged array
    merged.extend(left[left_index:])
    merged.extend(right[right_index:])

    return merged


# Example usage
arr = [64, 34, 25, 12, 22, 11, 90]
sorted_arr = merge_sort(arr)
print("Sorted array is:", sorted_arr)

```

**Explanation:**

Here's how the code works:

*   The `merge_sort` function takes an array as input and recursively divides it into two halves until each half contains a single element (since a single-element array is inherently sorted).
*   The `merge` function takes two sorted arrays and merges them into a single sorted array. It does this by comparing elements from the two arrays and appending the smaller one to the merged array. If one array is exhausted, the remaining elements from the other array are appended.
*   Finally, the sorted array is printed.

This implementation has a time complexity of O(n log n), where n is the number of elements in the input array.

## Reflection Step

Now, let's allow the LLM to reflect on its outputs by defining another system prompt. This system prompt will tell the LLM to act as Andrej Karpathy, computer scientist and Deep Learning wizard.

>To be honest, I don't think the fact of acting like Andrej Karpathy will influence the LLM outputs, but it was fun :)

<img src="../img/karpathy.png" alt="Alt text" width="500"/>

In [8]:
reflection_chat_history = [
    {
    "role": "system",
    "content": "You are Andrej Karpathy, an experienced computer scientist. You are tasked with generating critique and recommendations for the user's code",
    }
]

The user message, in this case,  is the essay generated in the previous step. We simply add the `mergesort_code` to the `reflection_chat_history`.

In [11]:
reflection_chat_history.append(
    {
        "role": "user",
        "content": mergesort_code
    }
)

Now, let's generate a critique to the Python code.

In [12]:
critique = client.chat.completions.create(
    messages=reflection_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

In [13]:
display_markdown(critique, raw=True)

Your implementation of the Merge Sort algorithm is correct and well-explained. Here's my critique and recommendations:

**Code Quality:**

* The code is well-structured, and the variable names are descriptive, making it easy to understand.
* You could consider adding docstrings to the `merge_sort` and `merge` functions to provide a brief description of what each function does.

**Performance:**

* As you mentioned, the time complexity of the Merge Sort algorithm is O(n log n), which is optimal for a comparison-based sorting algorithm.
* There's no significant performance improvement that can be made to this implementation.

**Best Practices:**

* You could consider handling edge cases, such as an empty input array or an array with a single element. Although your implementation already returns the original array for these cases, it's good practice to explicitly handle them.
* You might want to consider adding error handling for cases where the input is not a list or is a list of non-comparable elements.

**Code Style:**

* The code follows PEP 8 guidelines, which is great.
* You could consider using a consistent spacing between lines to improve readability.

**Suggestions:**

* Consider adding a `main` function to encapsulate the example usage and make the code more modular.
* You might want to add unit tests to verify the correctness of the implementation.

Overall, your implementation is correct, efficient, and well-explained. With a few minor adjustments, it can become even more robust and maintainable.

Finally, we just need to add this *critique* to the `generation_chat_history`, in this case, as the `user` role.

In [14]:
generation_chat_history.append(
    {
        "role": "user",
        "content": critique
    }
)

## Generation Step (II)

In [15]:
essay = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

In [16]:
display_markdown(essay, raw=True)

I appreciate your detailed critique and suggestions. Here's a revised version of the implementation that addresses your comments:

```Python
def merge_sort(arr):
    """
    Sorts an array using the Merge Sort algorithm.

    Args:
        arr (list): The input array to be sorted.

    Returns:
        list: The sorted array.
    """
    if not isinstance(arr, list):
        raise TypeError("Input must be a list")
    if len(arr) < 2:
        return arr
    middle = len(arr) // 2
    left_half = arr[:middle]
    right_half = arr[middle:]
    left_half = merge_sort(left_half)
    right_half = merge_sort(right_half)
    return merge(left_half, right_half)


def merge(left, right):
    """
    Merges two sorted arrays into a single sorted array.

    Args:
        left (list): The first sorted array.
        right (list): The second sorted array.

    Returns:
        list: The merged sorted array.
    """
    merged = []
    left_index = 0
    right_index = 0
    while left_index < len(left) and right_index < len(right):
        if left[left_index] < right[right_index]:
            merged.append(left[left_index])
            left_index += 1
        else:
            merged.append(right[right_index])
            right_index += 1
    merged.extend(left[left_index:])
    merged.extend(right[right_index:])
    return merged


def main():
    arr = [64, 34, 25, 12, 22, 11, 90]
    sorted_arr = merge_sort(arr)
    print("Sorted array is:", sorted_arr)


if __name__ == "__main__":
    main()
```

**Changes:**

* I added docstrings to the `merge_sort` and `merge` functions to provide a brief description of what each function does.
* I added a check to ensure that the input is a list, and raised a `TypeError` if it's not.
* I added a `main` function to encapsulate the example usage and make the code more modular.
* I used a consistent spacing between lines to improve readability.
* I didn't add unit tests in this revised version, but it's a great idea to include them in a real-world implementation.

Thank you again for your feedback!

## And the iteration starts again ...

After **Generation Step (II)** the corrected Python code will be received, once again, by Karpathy. Then, the LLM will reflect on the corrected output, suggesting further improvements and the loop will go, over and over for a number **n** of total iterations.

> There's another possibility. Suppose the Reflection step can't find any further improvement. In this case, we can tell the LLM to output some stop string, like "OK" or "Good" that means the process can be stopped. However, we are going to follow the first approach, that is, iterating for a fixed number of times.

## Implementing a class 

Now that you understand the underlying loop of the Reflection Agent, let's implement this agent as a class. 

In [17]:
from agentic_patterns import ReflectionAgent

In [18]:
agent = ReflectionAgent()

In [19]:
generation_system_prompt = """
You are a Python programmer tasked with generating high quality Python code.
Your task is to Generate the best content possible for the user's request. If the user provides critique,
respond with a revised version of your previous attempt."""

reflection_system_prompt = """
You are Andrej Karpathy, an experienced computer scientist. You are tasked with generating critique and recommendations 
for the user's code."""

prompt = """
Generate a Python implementation of the Merge Sort algorithm"""


In [20]:
final_response = agent.run(
    generation_system_prompt=generation_system_prompt,
    reflection_system_prompt=reflection_system_prompt,
    user_prompt=prompt,
    n_steps=3,
    verbose=1,
)


STEP 1/3



GENERATION

 Here is a Python implementation of the Merge Sort algorithm:

```python
def merge_sort(arr):
    """
    This function sorts an array using the Merge Sort algorithm.

    Args:
        arr (list): The input array to be sorted.

    Returns:
        list: The sorted array.
    """
    # If the array has only one element, it is already sorted
    if len(arr) <= 1:
        return arr

    # Find the middle of the array
    mid = len(arr) // 2

    # Divide the array into two halves
    left_half = arr[:mid]
    right_half = arr[mid:]

    # Recursively sort the two halves
    left_half = merge_sort(left_half)
    right_half = merge_sort(right_half)

    # Merge the sorted halves
    return merge(left_half, right_half)


def merge(left, right):
    """
    This function merges two sorted arrays into one sorted array.

    Args:
        left (list): The first sorted array.
        right (list): The second sorted array.

    Returns:
        list: The merged sorted 

## Final comparison

### First iteration

In [27]:
display_markdown(mergesort_code, raw=True)

Here's a Python implementation of the Merge Sort algorithm:

```Python
def merge_sort(arr):
    # If the input array contains fewer than two elements,
    # then return it as it's already sorted
    if len(arr) < 2:
        return arr

    # Find the middle point and devide it into two halves
    middle = len(arr) // 2
    left_half = arr[:middle]
    right_half = arr[middle:]

    # Recursively call merge_sort on the two halves
    left_half = merge_sort(left_half)
    right_half = merge_sort(right_half)

    # Merge the two sorted halves into a single sorted array
    return merge(left_half, right_half)


def merge(left, right):
    merged = []
    left_index = 0
    right_index = 0

    # Merge smaller elements first
    while left_index < len(left) and right_index < len(right):
        if left[left_index] < right[right_index]:
            merged.append(left[left_index])
            left_index += 1
        else:
            merged.append(right[right_index])
            right_index += 1

    # If there are remaining elements in either half, append them to the merged array
    merged.extend(left[left_index:])
    merged.extend(right[right_index:])

    return merged


# Example usage
arr = [64, 34, 25, 12, 22, 11, 90]
sorted_arr = merge_sort(arr)
print("Sorted array is:", sorted_arr)

```

**Explanation:**

Here's how the code works:

*   The `merge_sort` function takes an array as input and recursively divides it into two halves until each half contains a single element (since a single-element array is inherently sorted).
*   The `merge` function takes two sorted arrays and merges them into a single sorted array. It does this by comparing elements from the two arrays and appending the smaller one to the merged array. If one array is exhausted, the remaining elements from the other array are appended.
*   Finally, the sorted array is printed.

This implementation has a time complexity of O(n log n), where n is the number of elements in the input array.

### After 3 steps

In [29]:
display_markdown(final_response, raw=True)

Thank you for the feedback! I'm glad to hear that the code is looking good.

You're right, I apologize for the inconsistency in naming conventions. I'll make sure to stick to underscore notation throughout the code, as it's the convention in Python.

The error message improvement is a great suggestion. Providing a more informative error message will definitely help with debugging.

Adding a type hint of `-> None` to the `main` function is a good practice, as it indicates that the function doesn't return any value.

Regarding the docstring formatting, I'll make sure to add a blank line between the summary and the parameters section to improve readability.

Here's the updated code with the suggested changes:
```python
def merge_sort(arr: list) -> list:
    """
    Sorts an array using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    Parameters:
    arr (list): The input array to be sorted

    Returns:
    list: The sorted array
    """
    if not isinstance(arr, list):
        raise ValueError("Input must be a list, not {}".format(type(arr)))

    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left_half = arr[:mid]
    right_half = arr[mid:]

    left_half = merge_sort(left_half)
    right_half = merge_sort(right_half)

    return merge(left_half, right_half)


def merge(left: list, right: list) -> list:
    """
    Merges two sorted arrays into one sorted array.

    Parameters:
    left (list): The first sorted array
    right (list): The second sorted array

    Returns:
    list: The merged sorted array
    """
    merged = []
    left_index = 0
    right_index = 0

    while left_index < len(left) and right_index < len(right):
        if left[left_index] <= right[right_index]:
            merged.append(left[left_index])
            left_index += 1
        else:
            merged.append(right[right_index])
            right_index += 1

    merged.extend(left[left_index:])
    merged.extend(right[right_index:])

    return merged


def main() -> None:
    """
    Main function to test the merge sort algorithm.
    """
    arr = [64, 34, 25, 12, 22, 11, 90]
    sorted_arr = merge_sort(arr)
    print("Sorted array:", sorted_arr)


if __name__ == "__main__":
    main()
```
Thank you again for the feedback! I'll make sure to keep these suggestions in mind for future code reviews.